In [1]:
import sqlite3
from sqlite3 import Error

### Connect to SQLite DB-File

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
 
    return conn

In [3]:
database = r"data\\all-the-news.db"

conn = create_connection(database)

### And extract all Data

In [4]:
rows = conn.cursor().execute("SELECT id, date text, title text, content text, publication text FROM longform").fetchall()

In [5]:
conn.close()

### Store it as Pandas DataFrame

In [6]:
import pandas as pd

In [7]:
col_names = {0: "ID", 1: "Date", 2: "Headline", 3: "Content", 4: "Publication"}
df_news = pd.DataFrame(rows).rename(columns=col_names)

### Make some transformations

In [8]:
df_news.set_index(keys="ID",inplace=True)

In [9]:
before = df_news.shape[0]

to_drop = ["","[]"]
for value in col_names.values():
    if value != "ID":
        for item in to_drop:
            df_news = df_news[df_news[value] != item]
        df_news = df_news[df_news[value].notnull()]

        after = df_news.shape[0]
print("Rows dropped: {0:6d}".format(before - after))
print("Rows left: {0:6d}".format(after))

Rows dropped:  21748


In [10]:
df_news["Date"] = pd.to_datetime(df_news["Date"], errors="coerce")

### Store it as pickle file for faster loading

In [15]:
df_news.to_pickle("data/data_raw.pickle")